# Authoring a Multibody Simulation

This tutorial provides some tools to help you create a new scene description file that can be parsed into Drake's multibody physics engine (MultibodyPlant) and geometry engine (SceneGraph).

You can **duplicate this notebook**, and edit your file directly here in deepnote.  Just create a new file, or upload and existing file along with your art assets (e.g. obj and mtl files) to this project.

In [2]:
# Imports
import numpy as np
from pydrake.all import (
    AddMultibodyPlantSceneGraph, DiagramBuilder, FindResourceOrThrow, MeshcatVisualizerCpp,
    MeshcatVisualizerParams, Parser, RigidTransform, Role, RollPitchYaw, Simulator, StartMeshcat, JointSliders
)

In [3]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


## Inspecting a URDF / SDF using joint sliders

The most important formats for creating multibody scenarios in Drake are the "Universal Robot Description Format" (URDF) and the "Scene Description Format" (SDF)...

In [4]:
def inspector(filename, package_paths={}):
    meshcat.Delete()
    meshcat.DeleteAddedControls()
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)

    # Load the file into the plant/scene_graph.
    parser = Parser(plant)
    for name, directory in package_paths.items():
        parser.package_map().Add(name, directory)
    parser.AddModelFromFile(filename)
    plant.Finalize()

    # Add two visualizers, one to publish the "visual" geometry, and one to
    # publish the "collision" geometry.
    visual = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat,
        MeshcatVisualizerParams(role=Role.kPerception, prefix="visual"))
    collision = MeshcatVisualizerCpp.AddToBuilder(
        builder, scene_graph, meshcat,
        MeshcatVisualizerParams(role=Role.kProximity, prefix="collision"))
    # Disable the collision geometry at the start; it can be enabled by the
    # checkbox in the meshcat controls.
    meshcat.SetProperty("collision", "visible", False)

    sliders = builder.AddSystem(JointSliders(meshcat, plant))

    diagram = builder.Build()
    sliders.Run(diagram)

In [6]:

inspector("triple_cartpole.urdf")

INFO:drake:Press the 'Stop JointSliders' button in Meshcat to continue.


In [ ]:
filename = FindResourceOrThrow(
                "drake/manipulation/models/"
                "wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf")
inspector(filename)

## Creating (or porting) a new robot / object

The "inspector" method above will be very useful as we start to produce our own robot description files, or port description files over from another simulator.
- Link to URDF / SDF docs; possibly provide a little guidance here about which to prefer.
- Link to Drake custom tags.  (probably requires https://github.com/RobotLocomotion/drake/issues/16178)
- Best support (currently) is for OBJ files.  If you have some other format (stl, dae, ...), then consider using the open-source `MeshLab` to convert it.
- We often prefer simplified collision geometry (maybe link to my lecture notes on this)...
- When using OBJ files for collisions, Drake will take the convex hull (consider adding the convex tag to improve performance).  If you need a non-convex collision model, use a convex decomposition tool.  Many of the tools out there are just thin wrappers on VHACD.  Here is a wrapper that we often use for Drake: https://github.com/gizatt/convex_decomp_to_sdf 

TODO(russt): I can potentially provision this deepnote project with meshlab/vhacd/etc, or at very least provide the commands here in this notebook to do it.

## Creating (or porting) a new "scene" containing multiple robots/objects

- I believe we should only recommend using SDF with the `<include>` tag here.  Should we still point to the model directives?  or do we consider that already on the path to extinction?
- Welding objects...

In [ ]:
# One option to construct the scene is to load the objects programmatically.
def add_objects_programmatically(pladfnt):
    # Load a mug into the plant/scene_graph.
    parser = Parser(plant)
    parser.AddModelFromFile("/work/simple_mug/simple_mug.sdf")

    # Load a cracker box into the plant/scene_graph.
    cracker_box = FindResourceOrThrow(
                "drake/manipulation/models/"
                "ycb/sdf/003_cracker_box.sdf")
    parser.AddModelFromFile(cracker_box)

    # Load a desk into the plant/scene_graph.
    desk_model_instance = parser.AddModelFromFile("/work/long_table/long_table.sdf")
    desk = plant.GetBodyByName("long_table_body", desk_model_instance)
    # Weld the desk to the world.
    plant.WeldFrames(plant.world_frame(), desk.body_frame())

# An alternative to construct the scene is to build the scene in an SDF file.
# Notice that the desk has been welded inside the sdf file.
def add_objects_single_sdf(plant):
    parser = Parser(plant)
    parser.AddModelFromFile("/work/cluttered_desk.sdf")

def create_scene(dt, single_sdf):
    # Clean the plate for MeshCat.
    meshcat.Delete()
    meshcat.DeleteAddedControls()

    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=dt)

    # Populate the plant with the scene.
    if single_sdf:
        add_objects_single_sdf(plant)
    else:
        add_objects_programmatically(plant)

    # Finalize the plant after loading the scene.
    plant.Finalize()

    # Set the initial pose of the mug.
    mug = plant.GetBodyByName("mug_body")
    desk_frame = plant.GetFrameByName("table_top_center")
    plant_context = plant.CreateDefaultContext()
    X_WD = desk_frame.CalcPoseInWorld(plant_context)
    X_DM = RigidTransform(RollPitchYaw(np.asarray([45, -30, 0]) * np.pi / 180), p=[0,0,0.5])
    X_WM = X_WD.multiply(X_DM)
    plant.SetDefaultFreeBodyPose(mug, X_WM)
    # Set the initial pose of the cracker box.
    cracker_box = plant.GetBodyByName("base_link_cracker")
    X_DC = RigidTransform(p=[0,0,1.0])
    X_WC = X_WD.multiply(X_DC);
    plant.SetDefaultFreeBodyPose(cracker_box, X_WC)

    # Add visualizer to visualize the geometries.
    visualizer = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat,
        MeshcatVisualizerParams(role=Role.kPerception, prefix="visual"))

    diagram = builder.Build()
    return diagram, visualizer

## Running a simple simulation

Passive system (objects fall with gravity).  Did they do what you expect?

Adjusting the timestep.  Start small and increase it... 

In [ ]:
def initialize_simulation(diagram):
    simulator = Simulator(diagram)
    simulator.Initialize()
    simulator.set_target_realtime_rate(1.)
    return simulator

def run_simulation(dt):
    diagram, visualizer = create_scene(dt, single_sdf=True)
    simulator = initialize_simulation(diagram)
    visualizer.StartRecording()
    simulator.AdvanceTo(5.0)
    visualizer.PublishRecording()

# Run the simulation with a small timestep. Try gradually increasing it!
run_simulation(dt=0.0001)

## Debugging your MultibodyPlant/SceneGraph

Sometimes when you go to simulate, you get surprising results. 

If you're timestep is < 0.001, you probably have something poorly specified in your system creating stiffness.  This can happen, for instance, if you have bodies which don't have any inertial properties specified, but which are important to the dynamics.  (You don't need to specify the mass of a table if it's welded to the world, but if you have a movable coffee mug with zero mass, then the simulation is not yet fully specified.)

Hint: Does the mass/inertia of the mug seem reasonable?

## Next steps...

This tutorial helps you set up the physics and geometry engines (MultibodyPlant and SceneGraph), but most robotics simulations require more.  Next you might need to model the sensors, the low-level control system, and eventually even the high-level perception, planning, and control systems...

Could point to
- Examples (w/ teleop, etc) in manipulation notes
- Monte Carlo tools
- DrakeGymEnv

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8e7a0ac9-52e4-4b90-9852-01c6dc04ac04' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>